# DPO 微调：根据偏好引导LLM的输出
> [GenAI HW6: LLM Values Alignment](https://colab.research.google.com/drive/1d3zmkqo-ZmxrIOYWSe3vDD0za8tUPguu?usp=sharing#scrollTo=owGIuqdnRI8I) 中文镜像版
>
> 指导文章：[11. DPO 微调示例：根据人类偏好优化 LLM 大语言模型](https://github.com/Hoper-J/LLM-Guide-and-Demos-zh_CN/blob/master/Guide/11.%20DPO%20微调示例：根据人类偏好优化%20LLM%20大语言模型.md)

目标：学习如何使用带标签的偏好数据来对齐模型的行为。

大白话就是：微调。但这里使用的是 DPO 方法（之后会具体解释什么是 DPO）。

之前微调唐诗更像是一个传统的方法，使用问题和答案驱动的微调方式，数据标注成本很高。而 DPO 不同，它使用的是人类对文本的偏好对：即人类认为一对文本中哪个更好。

## 安装和导入一些必要的库

In [1]:
!pip install transformers
!pip install bitsandbytes
!pip install datasets
!pip install peft
!pip install trl
!pip install accelerate
!pip install tf-keras
!pip install numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 526.2 kB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 8.2 MB/s eta 0:00:00


In [2]:
import os
import re
import json

import torch
import pandas as pd
from tqdm.auto import tqdm

from datasets import Dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from trl import DPOConfig, DPOTrainer

## 加载数据集

In [3]:
!git clone https://github.com/Baiiiiiiiiii/GenAI_hw6_dataset.git

Cloning into 'GenAI_hw6_dataset'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), 4.06 KiB | 4.06 MiB/s, done.


In [4]:
with open("./GenAI_hw6_dataset/labelled_data.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("./GenAI_hw6_dataset/test_prompt.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

In [5]:
full_data[:5], test_data

([{'id': 1,
   'prompt': '日本動漫真人化是否有損原作形象？',
   'support': '真人化能夠呈現更真實的角色形象，提升原作魅力。',
   'oppose': '真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。'},
  {'id': 2,
   'prompt': '真人化是否能夠擴大動漫在全球的影響力？',
   'support': '真人化能夠讓更多非動漫迷接觸作品，擴大影響力。',
   'oppose': '真人化可能失去動漫的獨特風格，限制影響力擴大。'},
  {'id': 3,
   'prompt': '真人化是否能夠吸引新觀眾？',
   'support': '真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。',
   'oppose': '真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。'},
  {'id': 4,
   'prompt': '真人化是否能夠保留原作故事情節的精髓？',
   'support': '真人化有機會更深入挖掘原作故事，保留精髓。',
   'oppose': '真人化可能因為改編而失去原作故事的深度與精髓。'},
  {'id': 5,
   'prompt': '真人化是否能夠提升動漫產業的商業價值？',
   'support': '真人化能夠開拓更多商業機會，提升產業價值。',
   'oppose': '真人化可能讓觀眾對原作失去興趣，影響產業價值。'}],
 [{'id': 1, 'prompt': '真人化是否能改善日本漫畫的全球可及性？'},
  {'id': 2, 'prompt': '真人化如何影響年輕一代對日本漫畫的看法？'},
  {'id': 3, 'prompt': '真人化是否能提升原作漫畫的文學價值？'},
  {'id': 4, 'prompt': '真人化是否有助於保護和保存日本漫畫的傳統？'},
  {'id': 5, 'prompt': '真人化是否有助於提升日本漫畫行業的經濟效益？'},
  {'id': 6, 'prompt': '真人化如何影響日本漫畫原作者的創作動力？'},
  {'id': 7, 'prompt': '真人化是否對漫畫原作的忠實粉絲公平？'},
  {'id': 8, 'prompt': '真人化是否能夠促

## 加载模型

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    'MediaTek-Research/Breeze-7B-Instruct-v0_1',
    device_map='auto',
    trust_remote_code=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
)

# 本地取消下面的注释，然后注释上面
# model = AutoModelForCausalLM.from_pretrained(
#     'Breeze-7B-Instruct-v0_1',  # 替换为本地模型存储的实际路径
#     device_map='auto',
#     trust_remote_code=True,
#     local_files_only=True,  # 确保从本地加载
#     quantization_config=BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_compute_dtype=torch.bfloat16,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type='nf4'
#     )
# )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## 查看未经过微调的模型原始输出

In [7]:
tokenizer = AutoTokenizer.from_pretrained('MediaTek-Research/Breeze-7B-Instruct-v0_1')
# 本地取消下面的注释，然后注释上面
# tokenizer = AutoTokenizer.from_pretrained('Breeze-7B-Instruct-v0_1')
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

original_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f"Question {id}:\n{data['prompt']}")

    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config = GenerationConfig(
        do_sample=False,  # 随机挑选还是直接最大的
        max_new_tokens=200,
        pad_token_id=tokenizer.pad_token_id
    )
    # **inputs, 是一种将字典中的键值对解包为关键字参数的语法。
    output = model.generate(**inputs, generation_config=generation_config)
    output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]  # 第零个是不要多于20个词
    original_model_response.append(output_text)

    print(f"Response from original model:\n{output_text}\n")

tokenizer_config.json:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from original model:
真人化可能會影響年輕一代對日本漫畫的看法，使他們更容易接受和理解故事和角色，並吸引更多人關注和支持日本漫畫文化。然而，個人喜好和文化差異可能導致不同的影響。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from original model:
真人化可能有助於提高日本漫畫的知名度和吸引更多觀眾，但是否真正保護和保存傳統尚需視真人化作品是否尊重原作精神和文化價值。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？
Response from original model:
真人化可能有助於提升日本漫畫行業的經濟效益，因真人版電影或劇集可以吸引更多觀眾，增加收入來源。然而，成功與否取決於作品的品質、行銷策略和市場接受度。

Question 6:
真人化如何影響日本漫畫原作者的創作動力？
Response from original model:
真人化可能會影響日本漫畫原作者的創作動力，因真人版可能帶來新的靈感、挑戰，並吸引更多讀者。然而，個人感受和反應各異，有些作者可能因真人化而更投入創作，而其他人可能因個人喜好或對真人化看法不同而影響其動力。

Question 7:
真人化是否對漫畫原作的忠實粉絲公平？
Response from original model:
真人化可能會影響忠實的漫畫原作粉絲，因真人版可能有不同的故事改編、角色設定或表現方式，但個人喜好不同，仍有可能欣賞。

Question 8:
真

## 设置参数

你只需要修改这个模块，不需要改变其他的，除非你真的知道自己在做什么。

`support_ratio` 将反映你的偏好：

- 0 表示完全不支持（反对）真人化
- 1 表示完全支持真人化
- 0.1 表示 10% 支持真人化

In [16]:
num_epoch = 1       # 训练轮数
data_size = 50      # 用于训练的数据量
support_ratio = 0 # 偏好支持真人化的比例

## 准备训练数据

这里，我们将数据集分为支持（support）和反对（oppose）两部分，构建一个包含偏好对的训练数据集。

总共有 50 笔训练数据，当 support 设置为 0.1 时，前 50*0.1=5 笔训练资料的偏好将倾向于支持真人化，后 50-4=45 笔资料反对真人化。

In [17]:
# 选择部分数据用于训练
training_data = full_data[:data_size]

# 定义 support 数据集的大小，用于将一部分数据标记为“支持” (chosen)，另一部分标记为“反对” (rejected)
support_data_size = int(data_size * support_ratio)

# 为训练数据集准备数据
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]
position_list = ['support' for _ in range(support_data_size)] + ['oppose' for _ in range(data_size - support_data_size)]

# 创建训练数据集
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'position': position_list, 'chosen': chosen_list, 'rejected': rejected_list})
pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})

,prompt,position,preferred,non-preferred
0,<s>回覆請少於20字 [INST] 日本動漫真人化是否有損原作形象？ [/INST],oppose,真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。,真人化能夠呈現更真實的角色形象，提升原作魅力。
1,<s>回覆請少於20字 [INST] 真人化是否能夠擴大動漫在全球的影響力？ [/INST],oppose,真人化可能失去動漫的獨特風格，限制影響力擴大。,真人化能夠讓更多非動漫迷接觸作品，擴大影響力。
2,<s>回覆請少於20字 [INST] 真人化是否能夠吸引新觀眾？ [/INST],oppose,真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。,真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。
3,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作故事情節的精髓？ [/INST],oppose,真人化可能因為改編而失去原作故事的深度與精髓。,真人化有機會更深入挖掘原作故事，保留精髓。
4,<s>回覆請少於20字 [INST] 真人化是否能夠提升動漫產業的商業價值？ [/INST],oppose,真人化可能讓觀眾對原作失去興趣，影響產業價值。,真人化能夠開拓更多商業機會，提升產業價值。
5,<s>回覆請少於20字 [INST] 真人化是否能夠保持原作的文化特色？ [/INST],oppose,真人化可能因為文化差異而失去原作獨有的文化魅力。,真人化可以透過場景、服裝等元素保留文化特色。
6,<s>回覆請少於20字 [INST] 真人化是否能夠挑戰技術上的新突破？ [/INST],oppose,真人化可能因為技術限制而無法達到動畫中的視覺效果。,真人化促使技術創新，挑戰視覺效果上的新高度。
7,<s>回覆請少於20字 [INST] 真人化是否會受到演員選擇的爭議？ [/INST],oppose,演員選擇可能引起爭議，觀眾難以接受角色塑造。,演員選擇可因應市場需求，不必受限於動畫形象。
8,<s>回覆請少於20字 [INST] 真人化是否能夠提高動漫的社會認同度？ [/INST],oppose,真人化可能因為劇情改編而無法贏得社會認同。,真人化有機會讓更多人接受動漫，提高社會認同度。
9,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作角色的個性特色？ [/INST],oppose,真人化可能因演員演技或導演選擇而失去角色的原有特色。,真人化可以透過演員表現保留角色的個性特色。


## 训练

现在，我们进入训练阶段。首先，设置训练参数：

In [18]:
training_args = DPOConfig(
    output_dir='./',
    per_device_train_batch_size=1,
    num_train_epochs=num_epoch,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps = 1,
    warmup_ratio = 0.1,
    beta=0.1,
    report_to = 'none',

    # 显式声明以避免报错
    max_length=512,
    max_prompt_length=128,
    remove_unused_columns=False,
)

接下来，配置PEFT（Parameter-Efficient Fine-Tuning）：


In [19]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

然后，初始化DPO训练器：


In [20]:
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)

Extracting prompt from train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

开始训练：


In [21]:
dpo_trainer.train()

Step,Training Loss
1,0.693100
2,0.693100
3,0.564300
4,0.424300
5,0.303000
6,0.321200


TrainOutput(global_step=6, training_loss=0.49984591205914813, metrics={'train_runtime': 167.7303, 'train_samples_per_second': 0.298, 'train_steps_per_second': 0.036, 'total_flos': 0.0, 'train_loss': 0.49984591205914813, 'epoch': 0.96})

## 查看微调后的模型输出

In [22]:
trained_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f"Question {id}:\n{data['prompt']}")

    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config = GenerationConfig(
        do_sample=False,
        max_new_tokens=200,
        pad_token_id=tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    trained_model_response.append(output_text)

    print(f"Response from trained model:\n{output_text}\n")

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from trained model:
真人化可能會提高日本漫畫的全球可及性，但可能同時降低其文化獨特性。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from trained model:
真人化可能會影響年輕一代對日本漫畫的看法，因其可能改變原作者的原創設定、人物形象或故事情節，而影響讀者的觀感。此外，真人版可能無法完全表現漫畫的魅力，可能導致年輕讀者對日本漫畫的印象受到影響。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from trained model:
真人化可能會影響原作漫畫的文學價值，但結果取決於如何忠實地呈現原作的精神和故事。如果真人版能保持原作的核心概念、角色性格和故事情節，則文學價值可能保持或提升。然而，如果真人版偏離原作，則文學價值可能下降。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from trained model:
真人化可能會影響日本漫畫的傳統，但保護和保存傳統更需要從教育和文化傳承方面著手。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？
Response from trained model:
真人化可能會帶來短期的經濟效益，如票房收入和商品銷售，但長期來看，可能損害日本漫畫行業的品牌形象和原創性，影響市場競爭力。

Question 6:
真人化如何影響日本漫畫原作者的創作動力？
Response from trained model:
真人化可能影響日本漫畫原作者的創作動力，因其可能導致原作者的想法被限制或影響，同時也需要考慮到市場需求和觀眾喜好，這可能影響到原作者的原創性。

Question 7:
真人化是否對漫畫原作的忠實粉絲公平？
Response from trained model:
真人化可能會影響漫畫原作的忠實粉絲，因真人版可能與漫畫原作有差異，或無法完全表現原作的精神。但個人喜好不同，有些人可能更喜歡真人版，有些人可能更喜歡漫畫版。

Question 8:
真人化是否能夠促進日本漫畫的創新和多樣性？
Response from trained model:
真人化可能會限制日本

## 观察输出结果

In [23]:
model_response = []
print(f"num_epoch: {num_epoch}\ndata_size: {data_size}\nsupport_ratio: {support_ratio}\n")

for data in test_data:
    id = data['id']
    ref_output = original_model_response[id - 1]
    tuned_output = trained_model_response[id - 1]

    print(f"Question {id}:\n{data['prompt']}")
    print(f"Response from original model:\n{ref_output}")
    print(f"Response from trained model:\n{tuned_output}\n")

    model_response.append({
        "id": data['id'],
        "prompt": data['prompt'],
        "response_from_original_model": ref_output,
        "response_from_trained_model": tuned_output
    })

num_epoch: 1
data_size: 50
support_ratio: 0

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。
Response from trained model:
真人化可能會提高日本漫畫的全球可及性，但可能同時降低其文化獨特性。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from original model:
真人化可能會影響年輕一代對日本漫畫的看法，使他們更容易接受和理解故事和角色，並吸引更多人關注和支持日本漫畫文化。然而，個人喜好和文化差異可能導致不同的影響。
Response from trained model:
真人化可能會影響年輕一代對日本漫畫的看法，因其可能改變原作者的原創設定、人物形象或故事情節，而影響讀者的觀感。此外，真人版可能無法完全表現漫畫的魅力，可能導致年輕讀者對日本漫畫的印象受到影響。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。
Response from trained model:
真人化可能會影響原作漫畫的文學價值，但結果取決於如何忠實地呈現原作的精神和故事。如果真人版能保持原作的核心概念、角色性格和故事情節，則文學價值可能保持或提升。然而，如果真人版偏離原作，則文學價值可能下降。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from original model:
真人化可能有助於提高日本漫畫的知名度和吸引更多觀眾，但是否真正保護和保存傳統尚需視真人化作品是否尊重原作精神和文化價值。
Response from trained model:
真人化可能會影響日本漫畫的傳統，但保護和保存傳統更需要從教育和文化傳承方面著手。

Questi

## 获取 output 文件（如果需要的话取消注释）

In [ ]:
# with open(f"epoch-{num_epoch}_size-{data_size}_ratio-{support_ratio}.json", "w", encoding='UTF-8') as outfile:
#     json.dump(model_response, outfile, indent=4, ensure_ascii=False)